In [ ]:
%load_ext autotime
%load_ext autoreload
%autoreload 2

In [ ]:
import vulkan.cli.client as vulkan
from vulkan.cli.context import Context
from vulkan.data_source import HTTPSource, RetryPolicy
from vulkan.schemas import CachingOptions, DataSourceSpec

In [ ]:
ctx = Context()

In [ ]:
ctx.server_url = "http://localhost:6001"

In [ ]:
policy_id = vulkan.policy.create_policy(
    ctx,
    name="demo_policy",
    description="Demo Policy",
)

In [ ]:
api = "my-api"
version = "1.0"
MY_API_REF = f"{api}-v{version}"

vulkan.data.create_data_source(
    ctx,
    config=DataSourceSpec(
        name=MY_API_REF,
        description="Test data source",
        keys=[],
        source=HTTPSource(
            url="http://testdata:5000/",
            headers={"Content-Type": "application/json"},
            retry=RetryPolicy(
                max_retries=3,
                backoff_factor=1,
            ),
            body_schema={"tax_id": "str"},
            timeout=5,
        ),
        caching=CachingOptions(
            enabled=True,
            ttl=3600,
        ),
        metadata={
            "api": api,
            "version": version,
        },
    ),
)

In [ ]:
from vulkan.spec.dependency import INPUT_NODE, Dependency
from vulkan.spec.nodes import (
    BranchNode,
    DataInputNode,
    TerminateNode,
    TransformNode,
)
from vulkan.spec.policy import PolicyDefinition

In [ ]:
def _format_name(name: str) -> str:
    # Replace every character not in regex "^[A-Za-z0-9_]+$" with _
    return "".join(c if c.isalnum() or c == "_" else "_" for c in name)


# Format the data source name into a valid node name
MY_API_DATA_INPUT = _format_name(MY_API_REF)

In [ ]:
def prepare_request_body(inputs):
    return {"tax_id": inputs["tax_id"]}


body = TransformNode(
    name="api_request_body",
    func=prepare_request_body,
    dependencies={"inputs": Dependency(INPUT_NODE)},
)

api = DataInputNode(
    name=MY_API_DATA_INPUT,
    data_source=MY_API_REF,
    dependencies={"body": Dependency(body.name)},
)


def load_data(context, raw_data, **kwargs):
    import json

    context.log.info(f"Raw data: {raw_data}")
    data = json.loads(raw_data)
    context.log.info(f"Data loaded: {data} ({type(data)})")
    return data


process_data = TransformNode(
    name="post_process_data",
    func=load_data,
    dependencies={"raw_data": Dependency(api.name)},
)


def make_decision(context, scores, **kwargs):
    context.log.info(f"Scores: {scores}")
    if scores["scr"] > 600:
        return "approved"
    if scores["serasa"] > 800:
        return "analysis"
    return "denied"


decision = BranchNode(
    name="decision",
    func=make_decision,
    choices=["approved", "analysis", "denied"],
    dependencies={"scores": Dependency(process_data.name)},
)

approved = TerminateNode(
    name="approved",
    return_status="approved",
    dependencies={"condition": Dependency("decision", "approved")},
)

analysis = TerminateNode(
    name="analysis",
    return_status="analysis",
    dependencies={"condition": Dependency("decision", "analysis")},
)

denied = TerminateNode(
    name="denied",
    return_status="denied",
    dependencies={"condition": Dependency("decision", "denied")},
)

demo_policy = PolicyDefinition(
    nodes=[
        body,
        api,
        process_data,
        decision,
        approved,
        analysis,
        denied,
    ],
    input_schema={"tax_id": "str"},
)

In [ ]:
policy_version = vulkan.policy_version.create(
    ctx,
    policy_id=policy_id,
    version_name="v1",
    input_schema={"tax_id": "str"},
    spec=demo_policy.to_dict(),
)
policy_version

In [ ]:
vulkan.run.trigger_run_by_policy_version_id(
    ctx, policy_version["policy_version_id"], input_data={"tax_id": "1"}
)

In [ ]:
policy_version["policy_version_id"]